In [2]:
import json
import Database_connector
import pandas as pd
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import torch
import numpy as np
from Database_Settings import DB_SETTINGS
import Database_connector
from mysql.connector import (connection)
from PIL import Image

PICKLED_DATA_FOLDER = "Pickled_Data/"
PICKLED_PICTURES_FILE = "pictures_embeddings_resnet152_df.pkl"
PICKLED_OUTFIRS_FILE = "active_outfits_df.pkl"

PICTURES_DROP_COLUMNS = ["contentType", "status", "displayOrder", "sourceURL", "embeddings"]
#PICTURES_DROP_COLUMNS = []
OUTFITS_DROP_COLUMNS = ['owner', 'name', 'brand', 'isPublic', 'isDeleted', 'meta.validFrom', 'meta.validTo', "Outfit_size"]


# def get_active_outfits():
#     cnx = connection.MySQLConnection(**DB_SETTINGS)
#     db_connection = Database_connector.Db_Connection()
#     cursor = cnx.cursor(dictionary=True)
#     #tag_query = "SELECT * FROM Outfits WHERE (`Outfits`.`isPublic` = TRUE AND `Outfits`.`isDeleted` = FALSE AND `Outfits`.`meta.validTo` >= '9999-01-01 00:00:00')"
#     tag_query = "SELECT `Outfits`.`id` AS `id`, `Outfits`.`owner` AS `owner`, `Outfits`.`name` AS `name`, `Outfits`.`brand` AS `brand`, `Outfits`.`isPublic` AS `isPublic`, `Outfits`.`isDeleted` AS `isDeleted`, `Outfits`.`timeCreated` AS `timeCreated`, `Outfits`.`timeUpdated` AS `timeUpdated`, `Outfits`.`pricePerWeek` AS `pricePerWeek`, `Outfits`.`pricePerMonth` AS `pricePerMonth`, `Outfits`.`type` AS `type`, `Outfits`.`keywords` AS `keywords`, `Outfits`.`retailPrice` AS `retailPrice`, `Outfits`.`meta.validFrom` AS `meta.validFrom`, `Outfits`.`meta.validTo` AS `meta.validTo` FROM `Outfits` WHERE (`Outfits`.`isPublic` = TRUE AND `Outfits`.`isDeleted` = FALSE AND `Outfits`.`meta.validTo` >= '9999-01-01 00:00:00')"
#     cursor.execute(tag_query)
#     outfit_results = cursor.fetchall()

#     outfits_df_db = pd.DataFrame([list(order_dict.values()) for order_dict in outfit_results], columns=list(outfit_results[0].keys()))
#     return outfits_df_db

def get_pictures_df(path):
    pictures_df = pd.read_pickle(path)
    #pictures_df = pictures_df.drop(columns=PICTURES_DROP_COLUMNS)
    pictures_df = pictures_df.drop(columns=pictures_df.columns[2:])
    return pictures_df

def get_outfits_df(path):
    outfits_df = pd.read_pickle(path)
    outfits_df = outfits_df.drop(columns=OUTFITS_DROP_COLUMNS)
    return outfits_df

def prepare_data(pictures_df_path = PICKLED_DATA_FOLDER + PICKLED_PICTURES_FILE, outfits_df_path = PICKLED_DATA_FOLDER + PICKLED_OUTFIRS_FILE):
    pictures_df = get_pictures_df(pictures_df_path)
    outfits_df = get_outfits_df(outfits_df_path)
    return pictures_df, outfits_df

def picture_exists(picture_id, pictures_dir_path):
    return os.path.isfile(pictures_dir_path + os.sep + picture_id + ".jpg")

def find_missing_pictures(pictures_df, pictures_dir_path):
    pictures_df["file_exists"] = pictures_df["id"].apply(lambda x : picture_exists(x, pictures_dir_path))
    return pictures_df["file_exists"].value_counts()

In [7]:
pictures_df, outfits_df = prepare_data(pictures_df_path="../FREja_dataset_processing/pictures_embeddings_resnet152_df.pkl"
                                       , outfits_df_path="../FREja_dataset_processing/active_outfits_df.pkl")
pictures_df.head()

,id,owner
0,picture.00058abb53434872ae9bb4270ae21f8e,outfit.98f32aaf08bc4ff09c44e6e11e9199bc
1,picture.000cf715019a4a02aaf0dc479212195a,outfit.f11bdc25d87946f8b831b608bc8fb574
2,picture.0010c2e161154d6893734981d5455e76,outfit.9387d05b47f906c5
3,picture.00135ce8b1a04c5daa60cf7bdd99bcd5,outfit.12ae28ab0dc5494c98a1de2f8ce04b79
4,picture.001a58ea68da426384567b8cccc0c8a6,outfit.e989b8cb4a814d97b642e1cb326f47e6


In [8]:
PICTURES_DIR_PATH = "../FREja_dataset_processing/Filtered_pictures/"
DF_SPLITS = 200

DEVICE = "cuda:1"
EMBEDDINGS_SAVE_DIR = "Pickled_Data/Embeddings_final/"

if not os.path.isdir(EMBEDDINGS_SAVE_DIR):
    os.mkdir(EMBEDDINGS_SAVE_DIR)

In [7]:
find_missing_pictures(pictures_df, PICTURES_DIR_PATH)

True    18060
Name: file_exists, dtype: int64

In [9]:
from torchvision import transforms
import copy
from IPython.display import display
import ipywidgets
from tqdm.notebook import tqdm
from torchvision.io import read_image
from torchvision.io import read_image

def load_images_torchvision(pictures_df, n):
    image_ids = pictures_df["id"][:n]
    images = []
    for image_id in image_ids:
        image_path = PICTURES_DIR_PATH + image_id + ".jpg"
        pil_image = read_image(image_path)
        images.append(pil_image)
    return images

def load_image_torchvision(picture_id):
    image_path = PICTURES_DIR_PATH + picture_id + ".jpg"
    image = read_image(image_path)
    return image

def load_image(image_id):
    file_name = image_id + ".jpg"
    file_path = PICTURES_DIR_PATH + file_name
    image = read_image(file_path)
    return image

def image_to_embedding(model_e, image_id, d_widget, index_num):
    d_widget.value = f"Converting file number: {index_num}"
    pil_image = load_image(image_id)
    #input_tensor = preprocess(pil_image)
    input_tensor = pil_image
    # if torch.cuda.is_available():
    #     input_tensor = input_tensor.to('cuda')

    embedding = model_e(input_tensor)
    return embedding[0].squeeze().cpu()

def get_df_embeddings(pictures_df, embedding_model, preprocess):
    display_out = ipywidgets.HTML()
    display(display_out)

    if torch.cuda.is_available():
        embedding_model.to('cuda')

    split_dfs = np.array_split(pictures_df, DF_SPLITS)
    with torch.no_grad():
        for df_split in tqdm(split_dfs):
            df_split["embeddings"] = df_split.apply(lambda row: image_to_embedding(preprocess, embedding_model, row["id"], display_out, row.name), axis=1)
    embedding_df = pd.concat(split_dfs)
    return embedding_df


In [10]:
from tqdm import tqdm, tqdm_notebook
from torchvision.models import quantization
from torchvision import models
import torchvision

def load_image_series_torchvision(image_series):
    images = []
    for image_id in image_series:
        image_path = PICTURES_DIR_PATH + image_id + ".jpg"
        torch_image = read_image(image_path, mode=torchvision.io.ImageReadMode.RGB)
        images.append(torch_image)
    return images

class Embedding_Config():
    class exception_hack(Exception):
        pass

    
    class Parasite_Module(torch.nn.Module):
        def __init__(self, host_layer, include_host=True):
            super().__init__()
            self.host_layer = host_layer
            self.include_host = include_host

        def forward(self, x):
            if self.include_host:
                x = self.host_layer(x)
            raise Embedding_Config.exception_hack(x)

    def __init__(self):
        self.use_cuda = None
        
        self.weights = None
        self.transforms = None
        self.model = None
        self.model_save_name = None

    def load_model(self):
        raise NotImplementedError

    def prepare_model(self):
        model = self.load_model()
        if self.use_cuda:
            model.to(DEVICE)
        model.eval()
        return model

    def print_config_summary(self):
        print(f"PKL name: {self.model_save_name} | Model: {type(self.model).__name__} | Weights: {type(self.weights).__name__} | {self.weights.name}")

    def load_images(self, image_series):
        image_tensors = load_image_series_torchvision(image_series)
        formatted_images = [self.transforms(test_img).unsqueeze(0) for test_img in image_tensors]
        formatted_images = torch.vstack(formatted_images)
        if self.use_cuda:
            formatted_images = formatted_images.to(DEVICE)
        return formatted_images
    
    def get_embeddings(self, image_tensors):
        with torch.no_grad():
            try:
                embeddings = self.model(image_tensors)
            except self.exception_hack as e:
                embeddings = e.args[0]
        # Reduce to half precision to save space and load time
        forward_embeddings = embeddings.cpu()#embeddings.half().cpu()
        return forward_embeddings.half()

class resnet50Quantized_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = False
        
        self.weights = quantization.ResNet50_QuantizedWeights.IMAGENET1K_FBGEMM_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "resnet50_quantized.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = quantization.resnet50(weights=self.weights, quantize=True)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model
    
    def get_embeddings(self, image_tensors):
        with torch.no_grad():
            try:
                embeddings = self.model(image_tensors)
            except self.exception_hack as e:
                embeddings = e.args[0]
        embeddings = self.model.dequant(embeddings)
        return embeddings.cpu()

class resneXt101_64X4D_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = False
        
        self.weights = quantization.ResNeXt101_64X4D_QuantizedWeights.DEFAULT
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "resneXt101_64X4D_quantized.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = quantization.resnext101_64x4d(weights=self.weights, quantize=True)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model
    
    def get_embeddings(self, image_tensors):
        with torch.no_grad():
            try:
                embeddings = self.model(image_tensors)
            except self.exception_hack as e:
                embeddings = e.args[0]
        embeddings = self.model.dequant(embeddings)
        return embeddings.half().cpu()

class resnet50_V1_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "resnet50_v1.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class resnet50_V2_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "resnet50_v2.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class wide_resnet50_V1_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Wide_ResNet50_2_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "wide_resnet50_v1.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.wide_resnet50_2(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class wide_resnet50_V2_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Wide_ResNet50_2_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "wide_resnet50_v2.pkl"
        self.print_config_summary()
        
    
    def load_model(self):
        model = models.wide_resnet50_2(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class wide_resnet101_V1_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Wide_ResNet101_2_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "wide_resnet101_v1.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.wide_resnet101_2(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class wide_resnet101_V2_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Wide_ResNet101_2_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "wide_resnet101_v2.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.wide_resnet101_2(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

test_config = resnet50_V1_Config()
test_loaded_images = test_config.load_images(pictures_df["id"][:10])
test_embeddings = test_config.get_embeddings(test_loaded_images)
test_loaded_images.shape, test_embeddings.shape

PKL name: resnet50_v1.pkl | Model: ResNet | Weights: ResNet50_Weights | IMAGENET1K_V1


(torch.Size([10, 3, 224, 224]), torch.Size([10, 2048]))

In [12]:
class vgg16_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.VGG16_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "vgg16.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vgg16(weights=self.weights)
        model.classifier[3] = self.Parasite_Module(model.classifier[3], include_host=True)
        return model

class vgg16_bn_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.VGG16_BN_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "vgg16_bn.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vgg16_bn(weights=self.weights)
        model.classifier[3] = self.Parasite_Module(model.classifier[3], include_host=True)
        return model

class vgg19_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.VGG19_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "vgg19.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vgg19(weights=self.weights)
        model.classifier[3] = self.Parasite_Module(model.classifier[3], include_host=True)
        return model

class vgg19_bn_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.VGG19_BN_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "vgg19_bn.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vgg19_bn(weights=self.weights)
        model.classifier[3] = self.Parasite_Module(model.classifier[3], include_host=True)
        return model



In [13]:
class inception_v3_avgpooling_config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Inception_V3_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "inception_v3_acgpooling.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.inception_v3(weights=self.weights)
        model.avgpool = self.Parasite_Module(model.avgpool, include_host=False)
        #model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class inception_v3_config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Inception_V3_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "inception_v3.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.inception_v3(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

In [14]:
class ViT_H_14_E2E_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ViT_H_14_Weights.IMAGENET1K_SWAG_E2E_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vit_h_14(weights=self.weights)
        model.heads.head = self.Parasite_Module(model.heads.head, include_host=True)
        return model

class ViT_H_14_Linear_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ViT_H_14_Weights.IMAGENET1K_SWAG_LINEAR_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vit_h_14(weights=self.weights)
        model.heads.head = self.Parasite_Module(model.heads.head, include_host=True)
        return model

class ViT_B_16_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "ViT_B_16_Config.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vit_b_16(weights=self.weights)
        model.heads.head = self.Parasite_Module(model.heads.head, include_host=True)
        return model

class ViT_L_16_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ViT_L_16_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "ViT_L_16_Config.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vit_l_16(weights=self.weights)
        model.heads.head = self.Parasite_Module(model.heads.head, include_host=True)
        return model

class ViT_L_16_E2E_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ViT_L_16_Weights.IMAGENET1K_SWAG_E2E_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "ViT_L_16_E2E_Config.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vit_l_16(weights=self.weights)
        model.heads.head = self.Parasite_Module(model.heads.head, include_host=True)
        return model

class ViT_L_16_Linear_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ViT_L_16_Weights.IMAGENET1K_SWAG_LINEAR_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "ViT_L_16_Linear_Config.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vit_l_16(weights=self.weights)
        model.heads.head = self.Parasite_Module(model.heads.head, include_host=True)
        return model


class ViT_L_32_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ViT_L_32_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "ViT_L_32_Config.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vit_l_32(weights=self.weights)
        model.heads.head = self.Parasite_Module(model.heads.head, include_host=True)
        return model


In [15]:
from torchvision.models import detection, segmentation

def load_image_series_torchvision_tmp(image_series, resize_size=None):
    images = []
    for image_id in image_series:
        image_path = PICTURES_DIR_PATH + image_id + ".jpg"
        torch_image = read_image(image_path, mode=torchvision.io.ImageReadMode.RGB)
        if resize_size is not None:
            torch_image = torchvision.transforms.functional.resize(torch_image, resize_size)
        images.append(torch_image)
    return images

class fcn_resnet50_segmentation_classifier_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = segmentation.FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "fcn_resnet50_segmentation_classification.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = segmentation.fcn_resnet50(weights=self.weights)
        model.classifier[4] = self.Parasite_Module(model.classifier[4], include_host=True)
        #model.aux_classifier[4] = self.Parasite_Module(model.aux_classifier[4], include_host=True)
        return model

    def load_images(self, image_series):
        image_tensors = load_image_series_torchvision_tmp(image_series, resize_size=(512, 512))
        formatted_images = [self.transforms(test_img).unsqueeze(0) for test_img in image_tensors]
        formatted_images = torch.vstack(formatted_images)
        if self.use_cuda:
            formatted_images = formatted_images.to(DEVICE)
        return formatted_images
    
    def get_embeddings(self, image_tensors):
        with torch.no_grad():
            try:
                embeddings = self.model(image_tensors)
            except self.exception_hack as e:
                embeddings = e.args[0]
        return embeddings.cpu()

class fcn_resnet50_segmentation_aux_classifier_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = segmentation.FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "fcn_resnet50_segmentation_auxclassification.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = segmentation.fcn_resnet50(weights=self.weights)
        model.aux_classifier[4] = self.Parasite_Module(model.aux_classifier[4], include_host=True)
        return model

    def load_images(self, image_series):
        image_tensors = load_image_series_torchvision_tmp(image_series, resize_size=(512, 512))
        formatted_images = [self.transforms(test_img).unsqueeze(0) for test_img in image_tensors]
        formatted_images = torch.vstack(formatted_images)
        if self.use_cuda:
            formatted_images = formatted_images.to(DEVICE)
        return formatted_images
    
    def get_embeddings(self, image_tensors):
        with torch.no_grad():
            try:
                embeddings = self.model(image_tensors)
            except self.exception_hack as e:
                embeddings = e.args[0]
        return embeddings.cpu()



In [16]:
class Swin_B_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Swin_B_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.swin_b(weights=self.weights)
        model.head = self.Parasite_Module(model.head, include_host=False)
        return model

class Swin_S_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Swin_S_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.swin_s(weights=self.weights)
        model.head = self.Parasite_Module(model.head, include_host=False)
        return model

class Swin_T_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Swin_T_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.swin_t(weights=self.weights)
        model.head = self.Parasite_Module(model.head, include_host=False)
        return model

class Swin_V2_B_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Swin_V2_B_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.swin_v2_b(weights=self.weights)
        model.head = self.Parasite_Module(model.head, include_host=False)
        return model

class Swin_V2_S_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Swin_V2_S_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.swin_v2_s(weights=self.weights)
        model.head = self.Parasite_Module(model.head, include_host=False)
        return model

class Swin_V2_T_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Swin_V2_T_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.swin_v2_t(weights=self.weights)
        model.head = self.Parasite_Module(model.head, include_host=False)
        return model



In [21]:
class ViT_H_14_Linear_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ViT_H_14_Weights.IMAGENET1K_SWAG_LINEAR_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vit_h_14(weights=self.weights)
        model.heads.head = self.Parasite_Module(model.heads.head, include_host=False)
        return model

class RegNet_Y_32GF_V1_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.RegNet_Y_32GF_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.regnet_y_32gf(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class RegNet_Y_32GF_V2_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.RegNet_Y_32GF_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.regnet_y_32gf(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class RegNet_Y_32GF_E2E_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.RegNet_Y_32GF_Weights.IMAGENET1K_SWAG_E2E_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.regnet_y_32gf(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class RegNet_Y_32GF_Linear_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.RegNet_Y_32GF_Weights.IMAGENET1K_SWAG_LINEAR_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.regnet_y_32gf(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model



PKL name: resneXt101_64X4D_quantized.pkl | Model: QuantizableResNet | Weights: ResNeXt101_64X4D_QuantizedWeights | IMAGENET1K_FBGEMM_V1


(torch.Size([10, 3, 224, 224]), torch.Size([10, 2048]))

In [22]:
class resnet18_V1_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet18_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet18(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class resnet34_V1_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet34_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet34(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class resnet50_V1_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class resnet50_V2_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class resnet101_V1_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet101_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet101(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class resnet101_V2_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet101_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet101(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class resnet152_V1_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet152_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet152(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class resnet152_V2_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet152_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet152(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class resnet152_V2_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet152_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet152(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

# for test_class in [resnet18_V1_Config, resnet34_V1_Config, resnet50_V1_Config, resnet50_V2_Config, resnet101_V1_Config, resnet101_V2_Config, resnet152_V1_Config, resnet152_V2_Config]:
#     test_config_class = test_class()
#     test_loaded_images = test_config_class.load_images(pictures_df["id"][:10])
#     test_embeddings = test_config_class.get_embeddings(test_loaded_images)
#     test_loaded_images.shape, test_embeddings.shape

In [23]:
class AlexNet_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.AlexNet_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.alexnet(weights=self.weights)
        model.classifier[6] = self.Parasite_Module(model.classifier[6], include_host=True)
        return model

class AlexNet_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.AlexNet_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.alexnet(weights=self.weights)
        model.classifier[6] = self.Parasite_Module(model.classifier[6], include_host=False)
        return model

class RegNetY_16GF_E2E_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.RegNet_Y_128GF_Weights.IMAGENET1K_SWAG_E2E_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.regnet_y_128gf(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class RegNetY_16GF_Linear_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.RegNet_Y_128GF_Weights.IMAGENET1K_SWAG_LINEAR_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.regnet_y_128gf(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class vgg16_Config(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.VGG16_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = "vgg16.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vgg16(weights=self.weights)
        model.classifier[6] = self.Parasite_Module(model.classifier[6], include_host=True)
        return model

test_config_class = AlexNet_final()
test_loaded_images = test_config_class.load_images(pictures_df["id"][:10])
test_embeddings = test_config_class.get_embeddings(test_loaded_images)
test_loaded_images.shape, test_embeddings.shape

PKL name: AlexNet_final.pkl | Model: AlexNet | Weights: AlexNet_Weights | IMAGENET1K_V1


(torch.Size([10, 3, 224, 224]), torch.Size([10, 4096]))

In [9]:
class ConvNext_Large_v1(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ConvNeXt_Large_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.convnext_large(weights=self.weights)
        model.classifier[2] = self.Parasite_Module(model.classifier[2], include_host=True)
        return model

class ConvNext_Large_v1_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ConvNeXt_Large_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.convnext_large(weights=self.weights)
        model.classifier[2] = self.Parasite_Module(model.classifier[2], include_host=False)
        return model

class DenseNet201(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.DenseNet201_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.densenet201(weights=self.weights)
        model.classifier = self.Parasite_Module(model.classifier, include_host=True)
        return model

class DenseNet201_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.DenseNet201_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.densenet201(weights=self.weights)
        model.classifier = self.Parasite_Module(model.classifier, include_host=False)
        return model

class EfficientNet_V2_L(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.EfficientNet_V2_L_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.efficientnet_v2_l(weights=self.weights)
        model.classifier[1] = self.Parasite_Module(model.classifier[1], include_host=True)
        return model
    
class EfficientNet_V2_L_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.EfficientNet_V2_L_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.efficientnet_v2_l(weights=self.weights)
        model.classifier[1] = self.Parasite_Module(model.classifier[1], include_host=False)
        return model

class GoogLeNet(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.GoogLeNet_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.googlenet(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class GoogLeNet_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.GoogLeNet_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.googlenet(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class MobileNet_V3_Large(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.MobileNet_V3_Large_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.mobilenet_v3_large(weights=self.weights)
        model.classifier[3] = self.Parasite_Module(model.classifier[3], include_host=True)
        return model
    
class MobileNet_V3_Large_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.MobileNet_V3_Large_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.mobilenet_v3_large(weights=self.weights)
        model.classifier[3] = self.Parasite_Module(model.classifier[3], include_host=False)
        return model

class ResNeXt101_64X4D(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNeXt101_64X4D_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.resnext101_64x4d(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class ResNeXt101_64X4D_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNeXt101_64X4D_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.resnext101_64x4d(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class ShuffleNet_V2_X2(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ShuffleNet_V2_X2_0_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.shufflenet_v2_x2_0(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class ShuffleNet_V2_X2_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ShuffleNet_V2_X2_0_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.shufflenet_v2_x2_0(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class SqueezeNet1_1(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.SqueezeNet1_1_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.squeezenet1_1(weights=self.weights)
        model.classifier[3] = self.Parasite_Module(model.classifier[3], include_host=True)
        return model

class SqueezeNet1_1_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.SqueezeNet1_1_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.squeezenet1_1(weights=self.weights)
        model.classifier[3] = self.Parasite_Module(model.classifier[3], include_host=False)
        return model

class Swin_V2_B(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Swin_V2_B_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.swin_v2_b(weights=self.weights)
        model.head = self.Parasite_Module(model.head, include_host=True)
        return model

class Swin_V2_B_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Swin_V2_B_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.swin_v2_b(weights=self.weights)
        model.head = self.Parasite_Module(model.head, include_host=False)
        return model

class VGG19_BN(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.VGG19_BN_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vgg19_bn(weights=self.weights)
        model.classifier[6] = self.Parasite_Module(model.classifier[6], include_host=True)
        return model

class VGG19_BN_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.VGG19_BN_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vgg19_bn(weights=self.weights)
        model.classifier[6] = self.Parasite_Module(model.classifier[6], include_host=False)
        return model

class RegNet_Y_32GF(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.RegNet_Y_32GF_Weights.IMAGENET1K_SWAG_E2E_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.regnet_y_32gf(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class RegNet_Y_32GF_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.RegNet_Y_32GF_Weights.IMAGENET1K_SWAG_E2E_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.regnet_y_32gf(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class RegNet_X_32GF(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.RegNet_X_32GF_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.regnet_x_32gf(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class Wide_ResNet101_2(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Wide_ResNet101_2_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.wide_resnet101_2(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=True)
        return model

class Wide_ResNet101_2_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.Wide_ResNet101_2_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.wide_resnet101_2(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class MNASNet1_3(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.MNASNet1_3_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.mnasnet1_3(weights=self.weights)
        model.classifier[1] = self.Parasite_Module(model.classifier[1], include_host=True)
        return model

class MNASNet1_3_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.MNASNet1_3_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.mnasnet1_3(weights=self.weights)
        model.classifier[1] = self.Parasite_Module(model.classifier[1], include_host=False)
        return model

class MaxViT_T(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.MaxVit_T_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.maxvit_t(weights=self.weights)
        model.classifier[5] = self.Parasite_Module(model.classifier[5], include_host=True)
        return model

class MaxViT_T_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.MaxVit_T_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.maxvit_t(weights=self.weights)
        model.classifier[5] = self.Parasite_Module(model.classifier[5], include_host=False)
        return model

class ViT_H_14_E2E_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ViT_H_14_Weights.IMAGENET1K_SWAG_E2E_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
        
    def load_model(self):
        model = models.vit_h_14(weights=self.weights)
        model.heads.head = self.Parasite_Module(model.heads.head, include_host=False)
        return model

test_config_class = MaxViT_T()
test_loaded_images = test_config_class.load_images(pictures_df["id"][:10])
test_embeddings = test_config_class.get_embeddings(test_loaded_images)
print(test_loaded_images.shape, test_embeddings.shape)
test_config_class.model


/opt/conda/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


PKL name: MaxViT_T.pkl | Model: MaxVit | Weights: MaxVit_T_Weights | IMAGENET1K_V1
torch.Size([10, 3, 224, 224]) torch.Size([10, 1000])


MaxVit(
  (stem): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
      (2): GELU(approximate='none')
    )
    (1): Conv2dNormActivation(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (blocks): ModuleList(
    (0): MaxVitBlock(
      (layers): ModuleList(
        (0): MaxVitLayer(
          (layers): Sequential(
            (MBconv): MBConv(
              (proj): Sequential(
                (0): AvgPool2d(kernel_size=3, stride=2, padding=1)
                (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
              )
              (stochastic_depth): Identity()
              (layers): Sequential(
                (pre_norm): BatchNorm2d(64, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
                (conv_a): Conv2dNormActivation(
           

In [8]:
class resnet18_V1_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet18_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet18(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class resnet34_V1_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet34_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet34(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class resnet50_V1_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class resnet50_V2_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet50_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet50(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class resnet101_V1_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet101_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet101(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class resnet101_V2_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet101_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet101(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class resnet152_V1_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet152_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet152(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class resnet152_V2_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNet152_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnet152(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

resnet18_V1_final
PKL name: resnet18_V1_final.pkl | Model: ResNet | Weights: ResNet18_Weights | IMAGENET1K_V1
torch.Size([10, 3, 224, 224]) torch.Size([10, 512]) torch.float16 tensor(0.8271, dtype=torch.float16)
resnet18_V1_final.pkl
resnet34_V1_final
PKL name: resnet34_V1_final.pkl | Model: ResNet | Weights: ResNet34_Weights | IMAGENET1K_V1
torch.Size([10, 3, 224, 224]) torch.Size([10, 512]) torch.float16 tensor(0.8799, dtype=torch.float16)
resnet34_V1_final.pkl
resnet50_V1_final
PKL name: resnet50_V1_final.pkl | Model: ResNet | Weights: ResNet50_Weights | IMAGENET1K_V1
torch.Size([10, 3, 224, 224]) torch.Size([10, 2048]) torch.float16 tensor(0.4141, dtype=torch.float16)
resnet50_V1_final.pkl
resnet50_V2_final
PKL name: resnet50_V2_final.pkl | Model: ResNet | Weights: ResNet50_Weights | IMAGENET1K_V2
torch.Size([10, 3, 224, 224]) torch.Size([10, 2048]) torch.float16 tensor(0.0700, dtype=torch.float16)
resnet50_V2_final.pkl
resnet101_V1_final
PKL name: resnet101_V1_final.pkl | Model: R

In [16]:
class RegNet_Y_32GF_V1_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.RegNet_Y_32GF_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.regnet_y_32gf(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class RegNet_Y_32GF_V2_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.RegNet_Y_32GF_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.regnet_y_32gf(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model

class EfficientNet_B2_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.EfficientNet_B2_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.efficientnet_b2(weights=self.weights)
        model.classifier[1] = self.Parasite_Module(model.classifier[1], include_host=False)
        return model

class ViT_B_16_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ViT_B_16_Weights.IMAGENET1K_V1
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.vit_b_16(weights=self.weights)
        model.heads.head = self.Parasite_Module(model.heads.head, include_host=False)
        return model

class ResNeXt50_32X4D_final(Embedding_Config):
    def __init__(self):
        super().__init__()
        self.use_cuda = True

        self.weights = models.ResNeXt50_32X4D_Weights.IMAGENET1K_V2
        self.transforms = self.weights.transforms()
        self.model = self.prepare_model()
        self.model_save_name = f"{self.__class__.__name__}.pkl"
        self.print_config_summary()
    
    def load_model(self):
        model = models.resnext50_32x4d(weights=self.weights)
        model.fc = self.Parasite_Module(model.fc, include_host=False)
        return model


for test_class in [EfficientNet_B2_final, ViT_B_16_final, ResNeXt50_32X4D_final]:
    print(test_class.__name__)
    test_config_class = test_class()
    test_loaded_images = test_config_class.load_images(pictures_df["id"][:10])
    test_embeddings = test_config_class.get_embeddings(test_loaded_images)
    print(test_loaded_images.shape, test_embeddings.shape, test_embeddings.dtype, test_embeddings.mean())
    print(test_config_class.model_save_name)

EfficientNet_B2_final
PKL name: EfficientNet_B2_final.pkl | Model: EfficientNet | Weights: EfficientNet_B2_Weights | IMAGENET1K_V1
torch.Size([10, 3, 288, 288]) torch.Size([10, 1408]) torch.float16 tensor(0.0444, dtype=torch.float16)
EfficientNet_B2_final.pkl
ViT_B_16_final
PKL name: ViT_B_16_final.pkl | Model: VisionTransformer | Weights: ViT_B_16_Weights | IMAGENET1K_V1
torch.Size([10, 3, 224, 224]) torch.Size([10, 768]) torch.float16 tensor(0.0009, dtype=torch.float16)
ViT_B_16_final.pkl
ResNeXt50_32X4D_final
PKL name: ResNeXt50_32X4D_final.pkl | Model: ResNet | Weights: ResNeXt50_32X4D_Weights | IMAGENET1K_V2
torch.Size([10, 3, 224, 224]) torch.Size([10, 2048]) torch.float16 tensor(0.0837, dtype=torch.float16)
ResNeXt50_32X4D_final.pkl


In [17]:
import time

INFERENCE_TIME_LOG_FILE = "Pickled_Data/inference_time_log.txt"

def log_inference_time(config_class, embedding_shape, inference_time_array):
    with open(INFERENCE_TIME_LOG_FILE, "a") as f:
        f.write(f"\nModel: {config_class.model_save_name}, shape: {embedding_shape}\nmean: {inference_time_array.mean()}, values: {inference_time_array}")


def get_df_embeddings(base_df, config_class : Embedding_Config):
    # display_out = ipywidgets.HTML()
    # display(display_out)

    inference_time_log = []
    model_config = config_class()
    print(f"Generating embeddings from {type(model_config).__name__} to {model_config.model_save_name}...")
    split_dfs = np.array_split(base_df, DF_SPLITS)
    for df_split in tqdm(split_dfs):
        df_images = model_config.load_images(df_split["id"])

        start_time = time.time()
        df_embeddings = model_config.get_embeddings(df_images)
        embedding_shape = df_embeddings.shape
        df_embeddings = list(df_embeddings)
        inference_time_log.append(time.time() - start_time)

        df_split["embeddings"] = df_embeddings
    embedding_df = pd.concat(split_dfs)
    embedding_df.to_pickle(os.path.join(EMBEDDINGS_SAVE_DIR, model_config.model_save_name))
    log_inference_time(model_config, embedding_shape, np.array(inference_time_log))
    return embedding_df

#class_configs = [resneXt101_64X4D_Config, resnet50Quantized_Config]
#class_configs = [wide_resnet50_V1_Config, wide_resnet50_V2_Config, wide_resnet101_V1_Config, wide_resnet101_V2_Config, resneXt101_64X4D_Config, resnet50Quantized_Config, resnet50_V1_Config, resnet50_V2_Config]
#class_configs = [inception_v3_config, inception_v3_avgpooling_config]
#class_configs = [ViT_H_14_Config, ViT_B_16_Config, ViT_L_16_Config, ViT_L_16_E2E_Config, ViT_L_16_Linear_Config, ViT_L_32_Config]
#class_configs = [fcn_resnet50_segmentation_classifier_Config, fcn_resnet50_segmentation_aux_classifier_Config]
#class_configs = [Swin_V2_B_Config, Swin_V2_S_Config, Swin_V2_T_Config, Swin_B_Config, Swin_S_Config, Swin_T_Config]
#class_configs = [Swin_T_Config, inception_v3_avgpooling_config, ViT_H_14_Linear_Config, RegNet_Y_32GF_V1_Config, RegNet_Y_32GF_V2_Config, RegNet_Y_32GF_E2E_Config, RegNet_Y_32GF_Linear_Config]
#class_configs = [resnet18_V1_Config, resnet34_V1_Config, resnet50_V1_Config, resnet50_V2_Config, resnet101_V1_Config, resnet101_V2_Config, resnet152_V1_Config, resnet152_V2_Config]
#class_configs = [AlexNet_Config, RegNetY_16GF_E2E_Config, RegNetY_16GF_Linear_Config]
#class_configs = class_configs + [ViT_H_14_E2E_Config, ViT_H_14_Linear_Config, ViT_B_16_Config, ViT_B_16_Config, ViT_L_16_Config, ViT_L_16_E2E_Config, ViT_L_16_Linear_Config, ViT_L_32_Config]
#class_configs = [ConvNext_Large_v1, DenseNet201, EfficientNet_V2_L, MobileNet_V3_Large, ResNeXt101_64X4D, GoogLeNet, ShuffleNet_V2_X2, SqueezeNet1_1, Swin_V2_B]
#class_configs = [VGG19_BN, RegNet_X_32GF, RegNet_Y_32GF, Wide_ResNet101_2, MNASNet1_3]
#class_configs = [ViT_H_14_E2E_final, EfficientNet_V2_L_final, RegNet_Y_32GF_final]
#class_configs = [ConvNext_Large_v1_final, DenseNet201_final, EfficientNet_V2_L_final, MobileNet_V3_Large_final, ResNeXt101_64X4D_final, GoogLeNet_final, ShuffleNet_V2_X2_final, SqueezeNet1_1_final, Swin_V2_B_final]
#class_configs = [Swin_V2_B_final, VGG19_BN_final, RegNet_Y_32GF_final, Wide_ResNet101_2_final, MNASNet1_3_final]
#class_configs = [MaxViT_T_final]
#class_configs = [ViT_H_14_E2E_final]
#class_configs = [resnet18_V1_final, resnet34_V1_final, resnet50_V1_final, resnet50_V2_final, resnet101_V1_final, resnet101_V2_final, resnet152_V1_final, resnet152_V2_final]
class_configs = [RegNet_Y_32GF_V1_final, RegNet_Y_32GF_V2_final, EfficientNet_B2_final, ViT_B_16_final, ResNeXt50_32X4D_final]
for config_class in class_configs:
    get_df_embeddings(pictures_df, config_class)

PKL name: RegNet_Y_32GF_V1_final.pkl | Model: RegNet | Weights: RegNet_Y_32GF_Weights | IMAGENET1K_V1
Generating embeddings from RegNet_Y_32GF_V1_final to RegNet_Y_32GF_V1_final.pkl...


100%|██████████| 200/200 [3:11:59<00:00, 57.60s/it]   


PKL name: RegNet_Y_32GF_V2_final.pkl | Model: RegNet | Weights: RegNet_Y_32GF_Weights | IMAGENET1K_V1
Generating embeddings from RegNet_Y_32GF_V2_final to RegNet_Y_32GF_V2_final.pkl...


100%|██████████| 200/200 [3:01:06<00:00, 54.33s/it]   


PKL name: EfficientNet_B2_final.pkl | Model: EfficientNet | Weights: EfficientNet_B2_Weights | IMAGENET1K_V1
Generating embeddings from EfficientNet_B2_final to EfficientNet_B2_final.pkl...


100%|██████████| 200/200 [3:37:10<00:00, 65.15s/it]   


PKL name: ViT_B_16_final.pkl | Model: VisionTransformer | Weights: ViT_B_16_Weights | IMAGENET1K_V1
Generating embeddings from ViT_B_16_final to ViT_B_16_final.pkl...


100%|██████████| 200/200 [3:29:32<00:00, 62.86s/it]   
